In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


In [33]:
!pip install selenium webdriver-manager



In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [41]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer , AutoModelForSequenceClassification    #autotokenizer will convert string into sequence of series
import torch
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment') #loading pre-tained model
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [19]:
tokens=tokenizer.encode('it was kind of good',return_tensors='pt') #Change the sentence according to mood
print(tokens)

tensor([[  101, 10197, 10140, 16540, 10108, 12050,   102]])


In [20]:
print(tokenizer.decode(tokens[0]))

[CLS] it was kind of good [SEP]


In [21]:
result=model(tokens)
print(result.logits)
print(int(torch.argmax(result.logits))+1)

tensor([[-1.4118,  0.2249,  1.6721,  0.7788, -1.1994]],
       grad_fn=<AddmmBackward0>)
3


In [37]:
r=requests.get('https://www.yelp.com/biz/mejico-sydney-2') #Just Change to url to see any other reviews analysis for yelp
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*comment.*')
results=soup.find_all('p', {'class':regex})
reviews=[result.text for result in results]


In [38]:
print(results)

[<p class="comment__09f24__D0cxf y-css-1wfz87z"><span class="raw__09f24__T4Ezm" lang="en">Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.</span></p>, <p class="comment__09f24__D0cxf y-css-1wfz87z"><span class="raw__09f24__T4Ezm" lang="en">The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick

In [44]:
print(reviews)

["Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.", 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to chec

In [49]:
df=pd.DataFrame(np.array(reviews),columns=['review'])
df.tail()

,review
5,The food and service here was really good. It...
6,Visiting from Texas and decided to give this r...
7,"Great atmosphere, attentive service, solid mar..."
8,We came here on a Thursday night @ 5pm and by ...
9,The food is fresh and tasty. The scallop cevi...


In [50]:
df['review'].iloc[0]

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

In [51]:
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [53]:
sentiment_score(df['review'].iloc[1])

2

In [54]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [57]:
df

,review,sentiment
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,Don't come here expecting legit Mexican food b...,3
4,Out of all the restaurants that I tried in Syd...,5
5,The food and service here was really good. It...,5
6,Visiting from Texas and decided to give this r...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4


In [58]:
df['review'].iloc[7]

'Great atmosphere, attentive service, solid margs, and a Tasty menu. The Brisket Tacos were substantial and delicious. The corn ribs???  Fawgetaboutit!  Unreal.  Wanted to order another plate.'